https://github.com/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/GAN_for_MNIST_Tutorial.ipynb

#GAN 실습
- 논문 제목: Generative Adversarial Networks (NIPS 2014)
- 가장 기본적인 GAN 모델을 학습해보는 실습을 진행
- 학습 데이터셋: MNIST (1 X 28 X 28)

#필요한 라이브러리 불러오기
- 실습을 위한 PyTorch 라이브러리

In [1]:
import torch #파이토치 라이브러리 불러오기
import torch.nn as nn # 생성자 판별자의 아키테처 정의하기 위해

from torchvision import datasets #MNIST를 불러오기 위해
import torchvision.transforms as transforms #불러온 데이터셋을 우리가 의도한 방향으로 변형을 가해 전처리하기 위해
from torchvision.utils import save_image # 학습이 진행되는 과정에서 반복적으로 생성된 이미지를 출력하기 위해 

# 생성자(Generator) 및 판별자(Discriminator) 모델 정의

In [2]:
latent_dim = 100 # latent dimension을 뽑기 위한 noise distributer의 dimension??(본실습에서는 정규분포)

# 간단히 하나의 블록을 정의해서 원하는 블록을 원하는 만큼 쌓아서 사용할 수 있도록 함수를 정의
# 하나의 블록에서는 하나의 선형함수를 거친 뒤 배치정규화 시행
# 전체 생성자 모델은 이러한 여러 개의 블록을 연속적으로 가지게 됨

# 생성자(Generator) 클래스 정의
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        # 하나의 블록(block) 정의
        def block(input_dim, output_dim, normalize=True):
            layers = [nn.Linear(input_dim, output_dim)]
            if normalize:
                # 배치 정규화(batch normalization) 수행(차원 동일)
                layers.append(nn.BatchNorm1d(output_dim, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        # 생성자 모델은 연속적인 여러 개의 블록을 가짐
        self.model = nn.Sequential(
            *block(latent_dim, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, 1 * 28 * 28),# 결과적으로 1 * 28 * 28자리 하나의 MNIST data를 생성하도록 함
            nn.Tanh() #-1부터 1의 값을 가지기 위해
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), 1, 28, 28)
        return img

# forward함수를 확인하면 하나의 노이즈 벡터 z가 들어왔을 때 이를 모델에 놓고
# view함수를 이용해서 배치 사이즈 그리고 채널, 높이와 너비 차례대로 각각 이러한 차원을 가질 수 있도록해서
# 이미지 형태를 가질 수 있도록 만든다

## 판별자 클래스 정의

In [3]:
# 판별자(Discriminator) 클래스 정의
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(1 * 28 * 28, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )

##판별자 같은 경우는 반대로 한장의 이미지가 들어왔을 때 그 이미지를 판별하기 위해서
##여러개의 linear레이어와 activation function을 붙여서 결과적으로 sigmoid function으로 확률값을 내보낼 수 있도록 함

    # 이미지에 대한 판별 결과를 반환
    def forward(self, img):
        flattened = img.view(img.size(0), -1)
        output = self.model(flattened)

        return output

## 한장의 이미지가 들어왓을 때 먼저 슬레이트해서 하나의 벡터 형태로 나열한 뒤에 
## 그것을 모델에 넣어서 결과를 구할 수 있다.

# 데이터셋 불러오기
- 학습을 위한MINIST 데이터셋 불러오기

In [4]:
transforms_train = transforms.Compose([
    transforms.Resize(28),#크기는28x28
    transforms.ToTensor(), #파이토치의 tensor형태를 사용할 수 있도록 함
    transforms.Normalize([0.5], [0.5])
])

train_dataset = datasets.MNIST(root="./dataset", train=True, download=True, transform=transforms_train)
dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)

## 학습데이터만 불러온 뒤(train=True) 전체르 했던 함수를 이용해서 이미지를 변형하고
## 학습을 진행 할 때는 하나의 배치에 포함되어 있는 이미지가 128개가 되도록 한다.

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./dataset/MNIST/raw/train-images-idx3-ubyte.gz to ./dataset/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./dataset/MNIST/raw/train-labels-idx1-ubyte.gz to ./dataset/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to ./dataset/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./dataset/MNIST/raw



/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


#모델 학습 및 샘플링
- 학습을 위해 생성자와 판별자 모델을 초기화
- 적절한 하이퍼 파라미터를 설정

## 생성자(generator)와 판별자(discriminator) 초기화

In [5]:
generator = Generator()
discriminator = Discriminator() #생성자와 판변자 클래스를 이용해서 각각의 인스턴스를 초기화

generator.cuda() #cuda function을 들고와서 gpi로 올릴 수 있도록 해준다.
discriminator.cuda()

Discriminator(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): LeakyReLU(negative_slope=0.2, inplace=True)
    (4): Linear(in_features=256, out_features=1, bias=True)
    (5): Sigmoid()
  )
)

- ERROR발생시
  - 런타임> 런타임 유형변경 > GPU선택

## 손실함수, 학습률 설정

In [7]:
# 손실 함수(loss function)
adversarial_loss = nn.BCELoss()
adversarial_loss.cuda()

# 학습률(learning rate) 설정
lr = 0.0002

# 생성자와 판별자를 위한 최적화 함수
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))

- 모델을 학습하면서 주기적으로 샘플링하여 결과를 확인할 수 있다.

In [ ]:
import time

n_epochs = 200 # 학습의 횟수(epoch) 설정
sample_interval = 2000 # 몇 번의 배치(batch)마다 결과를 출력할 것인지 설정
start_time = time.time()

for epoch in range(n_epochs):
    for i, (imgs, _) in enumerate(dataloader):

        # 진짜(real) 이미지와 가짜(fake) 이미지에 대한 정답 레이블 생성
        real = torch.cuda.FloatTensor(imgs.size(0), 1).fill_(1.0) # 진짜(real): 1
        fake = torch.cuda.FloatTensor(imgs.size(0), 1).fill_(0.0) # 가짜(fake): 0

        real_imgs = imgs.cuda()
#생성자는 하나의 노이즈를 받아서 이미지를 만들 수 있기 때문에 이미지의 갯수만큼 이미지를 뽑아서
#그러한 노이즈들을 생성자에 넣어서 이미지를 만들 수 있도록 함
        """ 생성자(generator)를 학습합니다. """
        optimizer_G.zero_grad()

        # 랜덤 노이즈(noise) 샘플링
        z = torch.normal(mean=0, std=1, size=(imgs.shape[0], latent_dim)).cuda()

        # 이미지 생성
        generated_imgs = generator(z)

        # 생성자(generator)의 손실(loss) 값 계산
        g_loss = adversarial_loss(discriminator(generated_imgs), real)
## 생성자 입장에서는 자신이 만든 이미지가 real 이미지로 분류 될 수 있도록 학습을 해야하기 때문에
## 자기가 만든 이미지가 discriminator에 의해서 real이미지가 될 수 있도록 학습을 진행
        # 생성자(generator) 업데이트
        g_loss.backward()
        optimizer_G.step()

        """ 판별자(discriminator)를 학습합니다. """
        optimizer_D.zero_grad()

        # 판별자(discriminator)의 손실(loss) 값 계산
        real_loss = adversarial_loss(discriminator(real_imgs), real) #실제이미지를 실제 이미지라고 잘 분류하고
        fake_loss = adversarial_loss(discriminator(generated_imgs.detach()), fake) #생성자가 만든 가짜 이미지는 fake이라고 가짜 이미지로 분류하도록 함
        d_loss = (real_loss + fake_loss) / 2

        # 판별자(discriminator) 업데이트
        d_loss.backward()
        optimizer_D.step()

        done = epoch * len(dataloader) + i
        if done % sample_interval == 0:
            # 생성된 이미지 중에서 25개만 선택하여 5 X 5 격자 이미지에 출력
            save_image(generated_imgs.data[:25], f"{done}.png", nrow=5, normalize=True)

    # 하나의 epoch이 끝날 때마다 로그(log) 출력
    print(f"[Epoch {epoch}/{n_epochs}] [D loss: {d_loss.item():.6f}] [G loss: {g_loss.item():.6f}] [Elapsed time: {time.time() - start_time:.2f}s]")


- 생성된 이미지를 예시에 출력

In [ ]:
from IPython.display import Image

Image('92000.png')